### 1. Configuração Inicial e Carregamento de Dados
**Objetivo:** Preparar o ambiente de análise, importando as bibliotecas necessárias e carregando o conjunto de dados inicial.
Fonte de Dados: Banco de dados relacional.

In [111]:
# --- Importação de Bibliotecas Padrão e de Terceiros ---
import os
import re
import ast
import sys
import unidecode
import pandas as pd

# --- Configuração do Caminho do Projeto para Importações Locais ---
current_dir = os.getcwd()
project_root = os.path.abspath(os.path.join(current_dir, '..'))
if project_root not in sys.path:
    sys.path.append(project_root)

# --- Importação de Módulos e Funções Customizadas do Projeto ---
from src.utils.data_cleaning import normalize_text_columns
from src.utils.number_utils import br_to_float
from src.services.db_manager import fetch_data_from_db
from src.config import const

In [112]:
# Buscar os dados no banco de dados
df = fetch_data_from_db(const.consulta_sql)

### 2. Tratamento e Junção de Dados Demográficos

**Objetivo:** Adicionar o índice populacional de cada município ao DataFrame de trabalho.
**Fonte de Dados:** Arquivo com a população municipal, extraído do IBGE.

**Passos:**
1.  **Carregamento:** Leitura do arquivo de dados populacionais.
2.  **Tratamento:** Seleção das colunas de interesse (`código_ibge`, `populacao`,`UF`) e padronização dos tipos de dados.
3.  **Junção (`Merge`):** União do DataFrame de população com o DataFrame principal, usando o código IBGE como chave de ligação.

In [113]:
# Analisando os valores únicos na coluna 'municipioFornecedor'
df.municipioFornecedor.unique()

array(['PARANAIBA', 'SANTANA DE PARNAIBA', 'HORTOLANDIA', 'PORTO VELHO',
       'RIO BRANCO', 'MACEIO', 'FEIJO', 'OSASCO', 'RIBEIRAO PIRES',
       'BRASILIA', 'CRUZEIRO DO SUL', 'JARAGUA DO SUL', 'BELO HORIZONTE',
       'ITAJAI', 'VILA VELHA', 'PARNAIBA', 'BARUERI', 'PALMAS',
       'CARIACICA', 'TERESINA', 'SAO PAULO', 'RECIFE', 'OLINDA',
       'FORTALEZA', 'LONDRINA', 'TARAUACA', 'PALHOCA', 'MANAUS',
       'BOA VISTA', 'CAMPO GRANDE', 'SENA MADUREIRA', 'S?O LUIS',
       'SAO JOSE DOS CAMPOS', 'IBIPORA', 'PORTO WALTER', 'JORDAO',
       'MACAPA', 'CURITIBA', 'MARECHAL THAUMATURGO', 'CAJAMAR', 'NITEROI',
       'SANTA ROSA DO PURUS', '5300108', 'GOIANIA', 'RIO DE JANEIRO',
       'SAO LUIS', 'SÃO LUÍS', 'FLORIANOPOLIS', 'BELEM', 'VARZEA GRANDE',
       'ANAPOLIS', 'SALVADOR', 'SAO JOSE', 'SAO GABRIEL DA CACHOEIRA',
       'ILHEUS', 'SOROCABA', 'BRASÍLIA', 'UBERABA', 'CAMACARI',
       'BARREIRINHAS', 'MONTES CLAROS', 'NATAL', 'PIMENTA BUENO',
       'ANANINDEUA', 'DOURADOS', 'FOZ 

2.1 **Carregamento:**

In [114]:
# Carregar os dados demográficos dos municípios
mun = pd.read_csv(
    r"../data/raw/ibge_municipios.csv",
    sep=";",
    encoding="utf-8"
)

2.2 **Tratamento**

In [115]:
# Normalização: Deixa os nomes em Letra maiuscula e remove a acentuação
mun = normalize_text_columns(mun)
mun["COD. IBGE"] = mun["COD. UF"].astype(str).str.zfill(2) + mun["COD. MUNIC"].astype(str).str.zfill(5)
df['municipioFornecedor'] = df['municipioFornecedor'].apply(lambda x: unidecode.unidecode(x.upper()))

In [116]:
# Verificando quais municípios do DataFrame original não existem no DataFrame de municípios

mun['municipio']=mun['NOME DO MUNICÍPIO']

mun_set = set(mun['municipio'])
df2 = df.copy()
df2['existe_em_mun'] = df['municipioFornecedor'].apply(lambda x: x in mun_set)

nao_encontrados = df[~df2['existe_em_mun']]['municipioFornecedor'].unique()
print("Municípios não encontrados:")
print(nao_encontrados)

Municípios não encontrados:
['S?O LUIS' '5300108' 'CAMPO GRANDE (MS)' 'CUIABA (MT)'
 'SANTA CRUZ DO MONTE CASTELO']


In [117]:
# Correções manuais para alguns municípios conhecidos

ajustes = {
    'S?O LUIS': 'SAO LUIS',
    '5300108': 'BRASILIA',
    'CAMPO GRANDE (MS)': 'CAMPO GRANDE',
    'CUIABA (MT)': 'CUIABA',
    'SANTA CRUZ DO MONTE CASTELO': 'SANTA CRUZ DE MONTE CASTELO'
}

df['municipioFornecedor'] = df['municipioFornecedor'].replace(ajustes)


2.3 **Junção (`Merge`)**

In [118]:
df = df.merge(
    mun[['municipio', 'UF', 'POPULAÇÃO ESTIMADA', 'COD. IBGE']],
    left_on='municipioFornecedor',
    right_on='municipio',
    how='left'
)

## 3. Engenharia de Features e Ajustes Finais
**Objetivo:** Limpar, transformar e preparar as colunas do DataFrame principal para a fase de análise, garantindo a consistência e o formato correto dos dados.

**Passos:**
1.  **Tratamento das features:** Realizando tratamento inicial das colunas para melhor vizualização e melhor exploração dos dados.
2.  **Criando uma tabebela auxiliar:** Criar um DataFrame auxiliar (`df_itens`) onde cada linha representa um único item de uma nota fiscal, permitindo análises em nível de produto.
3.  **Separação das Features da coluna Auxiliar** filtramos as colunas mais importantes para a tabela auxiliar

3.1 **Tratamento das features:**

In [119]:
# Renomear colunas para nomes mais amigáveis
df = df.rename(columns={
    'id': 'ID',
    'orgaoDestinatario': 'ORGAO',
    'nomeFornecedor': 'FORNECEDOR',
    'cnpjFornecedor': 'CNPJ',
    'municipioFornecedor': 'MUNICIPIO',
    "valorNotaFiscal" : 'VALOR_NF',
    'itensNotaFiscal': 'ITENS',
    'tipoEventoMaisRecente': 'TIPO_EVENTO',
    'dataEmissao': 'DATA',
    'eventosNotaFiscal': 'EVENTOS',
    'chaveNotaFiscal': 'CHAVE_NF',
    'municipio': 'MUNICIPIO_MUN',
    'UF': 'UF',
    'POPULAÇÃO ESTIMADA': 'POPULACAO',
    'COD. IBGE': 'COD_IBGE'
})

In [120]:
# Remove tudo que não é número
df['CNPJ'] = df['CNPJ'].apply(lambda x: re.sub(r'\D', '', str(x)))

In [121]:
# Converter VALOR_NF de string para float
df['VALOR_NF'] = df['VALOR_NF'].str.replace('.', '', regex=False)  # remove separador de milhar
df['VALOR_NF'] = df['VALOR_NF'].str.replace(',', '.', regex=False)  # converte vírgula decimal para ponto
df['VALOR_NF'] = pd.to_numeric(df['VALOR_NF'], errors='coerce')


In [122]:
# Garantir que POPULACAO seja numérico
df['POPULACAO'] = pd.to_numeric(df['POPULACAO'], errors='coerce')

In [123]:
# Remover linhas inválidas
df = df.dropna(subset=['VALOR_NF', 'POPULACAO'])

In [124]:
# Converter DATA para o formato datetime
df['DATA'] = pd.to_datetime(df['DATA'], dayfirst=True)

# Criar coluna Ano-Mês
df['ANO_MES'] = df['DATA'].dt.to_period('M')

3.2 **Criando uma tabebela auxiliar:**

In [125]:
# Converter a coluna ITENS de string para lista de dicionários
df['ITENS'] = df['ITENS'].apply(ast.literal_eval)

# Explodir a lista em várias linhas (cada item vira uma linha)
df_itens = df.explode('ITENS')

# Transformar o dicionário de cada item em colunas
df_itens = pd.concat([df_itens.drop(columns=['ITENS']),
                      df_itens['ITENS'].apply(pd.Series)], axis=1)

# Selecionar apenas as colunas que você quer manter + CHAVE_NF
colunas_desejadas = ['CHAVE_NF', 'descricaoProdutoServico', 'codigoNcmSh', 
                     'ncmSh', 'cfop', 'quantidade', 'unidade', 'valorUnitario', 'valor']

df_itens_aux = df_itens[colunas_desejadas]

df_itens['quantidade'] = df_itens['quantidade'].apply(br_to_float)
df_itens['valorUnitario'] = df_itens['valorUnitario'].apply(br_to_float)
df_itens['valor'] = df_itens['valor'].apply(br_to_float)



3.3 **Separação das Features da coluna Auxiliar**

In [126]:
colunas_desejadas = ['descricaoProdutoServico', 'CHAVE_NF', 'codigoNcmSh', 'ncmSh', 
                     'cfop', 'quantidade', 'unidade', 'valorUnitario', 'valor']

df_itens = df_itens[colunas_desejadas]

In [131]:
df_itens.sample(2)

,descricaoProdutoServico,CHAVE_NF,codigoNcmSh,ncmSh,cfop,quantidade,unidade,valorUnitario,valor
3548,SUPER SAFETY LUVA SUPER LATEX TAM 7,12230929296383000179550020000001131016664015,40151900,"Outras luvas de borracha vulcanizada, não endu...",5102,15.0,PARES,14.90,223.5
1066,HF.ALHO SUPER ALHO TRITURADO POTE,35230775315333025012550010000424171751243482,7032090,"Alhos, frescos ou refrigerados, exceto para se...",5102,5.0,UND9,2.78,13.9


## 4. Exportação dos Dataframes para o formato .csv
**Informativo:** Agora que relizado o pré-processamento dos dados, os mesmo serão exportados para o formato .csv para fins da Análise Exploratória dos Dados

In [ ]:
# Definir diretório de saída (pasta processed dentro de data)
output_dir = os.path.join(project_root, "data", "processed")


# Exportar df
df.to_csv(
    os.path.join(output_dir, "notas_fiscais.csv"),
    index=False,
    sep=";", 
    encoding="utf-8"
)

# Exportar df_itens detalhado
df_itens.to_csv(
    os.path.join(output_dir, "itens_notas_fiscais.csv"),
    index=False,
    sep=";", 
    encoding="utf-8"
)